# Analiza obwodu elektrycznego - nadokreślony układ równań

importujemy potrzebne biblioteki

In [925]:
import numpy as np
import networkx as nx
from pyvis.network import Network
from collections import deque


wczytujemy graf z pliku

In [926]:
def load_from_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        G = nx.Graph()
        for line in lines:
            u, v, resistance = line.split()
            G.add_edge(int(u), int(v), res=float(resistance))
    return G

Znajdujemy natężenia prądów w obwodzie metodą potencjałów węzłowych
W tym celu musimy dla każdego węzła obliczyć jego potencjał elektryczny na podstawie prawa Kirchhoffa dla węzłów oraz prawa Ohma dla gałęzi.
Następnie dla każdej gałęzi obliczamy natężenie prądu na podstawie różnicy potencjałów oraz rezystancji.

n - liczba węzłów
m - liczba gałęzi
Macierz konduktancji A:
(dla każdego sąsiada węzła i)
- A[i][i] = 1 / rezystancja, czyli  sumujemy przewodności wszystkich gałęzi wychodzących z węzła.
- A[i][j] = -1 / rezystancja, czyli odejmujemy wpływ innych węzłów

Wektor wymuszeń b:
- b[i] = E, dla węzła źródłowego
- b[i] = 0, dla pozostałych węzłów

Rozwiązujemy układ równań A * V = b, gdzie V to wektor potencjałów węzłowych
Następnie obliczamy różnicę potencjałów dla każdej gałęzi i na jej podstawie natężenie prądu ze wzoru I = V_diff / rezystancja

In [927]:

def solve_circuit(file, s, t, E):
    G = load_from_file(file)
    nodes = list(G.nodes)
    
    masa = t  # Ustalony węzeł odniesienia
    n = len(nodes)

    # Macierz konduktancji i wektor wymuszeń
    A = np.zeros((n, n))
    b = np.zeros(n)

    for i, node in enumerate(nodes):
        if node == masa or node == s:
            A[i][i] = 1
        else:
            for neighbor in G.neighbors(node):
                j = nodes.index(neighbor)
                res = G.get_edge_data(node, neighbor)['res']
                A[i][j] -= 1 / res
                A[i][i] += 1 / res
                
    # Wymuszenie źródła napięciowego między s a t
    b[nodes.index(s)] = E 

    # Rozwiązanie układu równań
    V = np.linalg.solve(A, b)

    # Przypisanie potencjałów do węzłów
    for i, node in enumerate(nodes):
        G.nodes[node]['V'] = V[i]

    # Obliczenie prądów w gałęziach
    for u, v, data in G.edges(data=True):
        V_diff = V[nodes.index(u)] - V[nodes.index(v)]
        I = V_diff / data['res']
        G[u][v]['V'] = V_diff
        G[u][v]['I'] = I

    return G

Rysowanie obwodu
ustawiamy kolor zielony dla niskiego prądu i czerwony dla wysokiego
strzalka wskazuje kierunek prądu, ktory ustawiany jest na podstawie różnicy potencjałów

In [928]:
low_color = [0, 255, 0]
high_color = [255, 0, 0]
def draw_circuit(G, name="circuit"):
    # print(G.edges(data=True))
    net = Network(notebook=True, cdn_resources="remote", height="800px", width="100%")
    for node in G.nodes:
        net.add_node(node, label=str(node) + f" V: {G.nodes[node]['V']:.2f}")
        # net.add_node(node, label=str(node))
    for edge in G.edges(data=True):
        u, v, data = edge
        color = f"rgb({int(low_color[0] + (high_color[0] - low_color[0]) * (abs(data['I'] )/ 10))}, {int(low_color[1] + (high_color[1] - low_color[1]) * (abs(data['I'] )/ 10))}, 0)"
        # specjalne traktowanie źródła
        if 'res' in data and data['res'] == 0:
            net.add_edge(u, v, label=f"I: {abs(data['I']):.2f}A, R: {data['res']:.2f} ", color=color, width=5, arrows='from')
        elif 'I' in data:
            # skierowanie strzałki
            direction = 'to' if data['I'] > 0 else 'from'
            # net.add_edge(u, v, label=f"I: {data['I']:.2f}A, R: {data['res']:.2f} V: {data['V']:.2f}", color=color, width=5, arrows='from')
            net.add_edge(u, v, label=f"I: {abs(data['I']):.2f}A, R: {data['res']:.2f} ", color=color, width=5, arrows=direction)
        else:
            print("No I")
            net.add_edge(u, v, label=f"R: {data['res']:.2f}")
    net.toggle_physics(True)
    net.set_options("""
    var options = {
        "physics": {
            "barnesHut": {
                "gravitationalConstant": -8000,
                "centralGravity": 0.3,
                "springLength": 400,
                "springConstant": 0.04,
                "damping": 0.09
            },
            "minVelocity": 0.75
        }
    }
    """)
    net.show(f'visualization/{name}.html')


Testowanie układu z wykorzystaniem prawa Kirchoffa który mówi, że suma prądów wchodzących do węzła jest równa sumie prądów wychodzących z węzła

In [929]:
EPS = 1e-10

def check_first_kirchoff_law(G,s,t):
    verts = [0]*len(G.nodes)
    for e in G.edges:
        i = G[e[0]][e[1]]['I']
        verts[e[0]] -= i
        verts[e[1]] += i
        
    for i, curr in enumerate(verts):
        if i == s or i == t:
            continue
        if abs(curr) > EPS:
            return False
    
    return True


In [930]:
def TEST(Gdir, s, t):
    print("I Kirchoff law:", "PASSED" if check_first_kirchoff_law(Gdir,s,t) else "FAILED!!")

In [931]:
np.random.seed(33)
def save_to_file(G, file_path):
    with open(file_path, 'w') as file:
        for u, v in G.edges:
            file.write(f"{u} {v} {np.random.randint(1,10)}\n")

Generowanie grafów

In [932]:
def generate_graphs_to_files():
    # erdos_renyi_graph
    erdos = nx.erdos_renyi_graph(20, 0.5, seed=33)
    save_to_file(erdos, "graphs/erdos.txt")
    # regular_graph
    regular = nx.random_regular_graph(3, 16, seed=33)
    save_to_file(regular, "graphs/regular.txt")
    # connected_graph
    connected = nx.disjoint_union(nx.erdos_renyi_graph(10, 0.5, seed=33), nx.erdos_renyi_graph(10, 0.5, seed=33))
    connected.add_edge(5, 12, res=1)
    save_to_file(connected, "graphs/connected.txt")
    # grid_graph
    grid = nx.convert_node_labels_to_integers(nx.grid_2d_graph(5, 5))
    save_to_file(grid, "graphs/grid.txt")
    # small_world_graph
    small_world = nx.watts_strogatz_graph(20, 4, 0.5, seed=33)
    save_to_file(small_world, "graphs/small_world.txt")
generate_graphs_to_files()

In [933]:
def add_source_edge(G, s, t):
    A_sum = 0
    for neighbor in G.neighbors(s):
        A_sum +=  G.get_edge_data(s, neighbor)['I']
        
    G.add_edge(s, t, res=0, I=A_sum)

Testowanie

In [934]:
erdos= solve_circuit("graphs/erdos.txt", 0, 15,100)
draw_circuit(erdos, "erdos")
TEST(erdos,  0, 15)

regular = solve_circuit("graphs/regular.txt", 0, 15,100)
draw_circuit(regular, "regular")
TEST(regular,  0, 15)

connected = solve_circuit("graphs/connected.txt", 0, 15,100)
add_source_edge(connected, 0,15)
draw_circuit(connected, "connected")
TEST(connected,  0, 15)

grid = solve_circuit("graphs/grid.txt", 0, 24,100)
draw_circuit(grid, "grid")
TEST(grid,  0, 24)

small_world = solve_circuit("graphs/small_world.txt", 0, 15,100)
draw_circuit(small_world, "small_world")
TEST(small_world,  0, 15)


visualization/erdos.html
I Kirchoff law: PASSED
visualization/regular.html
I Kirchoff law: PASSED
visualization/connected.html
I Kirchoff law: PASSED
visualization/grid.html
I Kirchoff law: PASSED
visualization/small_world.html
I Kirchoff law: PASSED
